In [1]:
import pandas as pd
import random
import numpy as np
import glob
import os
import pickle

startDate = "20200427"
endDate = "20200428"
readPath = r'\\192.168.10.30\Kevin_zhenyu\rawData\***'
dataPathLs = np.array(glob.glob(readPath))
dateLs = np.array([os.path.basename(i).split('_')[1] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
opath = []
tpath = []
for i in dataPathLs:
    opath.append(glob.glob(os.path.join(i, "mdOrderLog_***")))
    tpath.append(glob.glob(os.path.join(i, "mdTradeLog_***")))
opath1 = opath[::2]
opath2 = opath[1::2]
tpath1 = tpath[::2]
tpath2 = tpath[1::2]
print(opath1)
print(opath2)
print(tpath1)
print(tpath2)
F1 = open("\\\\192.168.10.30\\Kevin_zhenyu\\index\\weight_table_CSI1000.pkl", 'rb')
df1 = pickle.load(F1)


for i in range(len(opath1)):
    TradeLog = pd.read_csv(tpath2[i][0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
    OrderLog = pd.read_csv(opath2[i][0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "Side", "OrderType", "Price",
                                                 "OrderQty"]]
    TradeLog1 = pd.read_csv(tpath1[i][0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
    OrderLog1 = pd.read_csv(opath1[i][0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "Side", "OrderType", "Price",
                                                 "OrderQty"]]
    stocks = OrderLog1["SecurityID"].unique()
    
    d_ate = os.path.basename(tpath1[i][0]).split('_')[1]
    d_ate = d_ate[:4] + '-' + d_ate[4:6] + '-' + d_ate[6:]
    ID_list = df1[df1["date"] == d_ate]["ID"]
    len1 = ID_list[ID_list.str[:2] == "SH"].shape[0]
    len2 = ID_list[ID_list.str[:2] == "SZ"].shape[0]
    print(len1)
    print(len2)
    ID_list = ID_list.str[2:].astype(int)
    
    l1 = stocks[:1000]
    OrderLogSZ = OrderLog[OrderLog["SecurityID"].isin(l1)]
    OrderLogSZ1 = OrderLog1[OrderLog1["SecurityID"].isin(l1)]
    del OrderLog
    del OrderLog1
    OrderLogSZ["idx"] = np.where(OrderLogSZ["SecurityID"].isin(ID_list.values), 1, 0)
    OrderLogSZ1["idx"] = np.where(OrderLogSZ1["SecurityID"].isin(ID_list.values), 1, 0)
    print(len(OrderLogSZ["SecurityID"].unique()))
    print(len(OrderLogSZ1["SecurityID"].unique()))
    TradeLogSZ = TradeLog[TradeLog["SecurityID"].isin(l1)]
    TradeLogSZ1 = TradeLog1[TradeLog1["SecurityID"].isin(l1)]
    TradeLogSZ["idx"] = np.where(TradeLogSZ["SecurityID"].isin(ID_list.values), 1, 0)
    TradeLogSZ1["idx"] = np.where(TradeLogSZ1["SecurityID"].isin(ID_list.values), 1, 0)
    print(len(TradeLogSZ["SecurityID"].unique()))
    print(len(TradeLogSZ1["SecurityID"].unique()))
    del TradeLog
    del TradeLog1
    OrderLogSZ["OrderType"] = OrderLogSZ["OrderType"].apply(lambda x: str(x))
    OrderLogSZ1["OrderType"] = OrderLogSZ1["OrderType"].apply(lambda x: str(x))
    
    # TradeLog check
    TradeLogSZ["ExecType"] = TradeLogSZ["ExecType"].apply(lambda x: str(x))
    TradeLogSZ1["ExecType"] = TradeLogSZ1["ExecType"].apply(lambda x: str(x))
    columns = ["TransactTime","ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag","TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum","OfferApplSeqNum"]
    re = pd.merge(TradeLogSZ, TradeLogSZ1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
    n1 = re["sequenceNo_x"].count()
    n2 = re["sequenceNo_y"].count()
    len1 = len(re)
    display(os.path.basename(os.path.basename(tpath2[i][0]).split("_")[1] + "_TradeLog Check_first 1000:"))
    print(n1)
    print(n2)
    print(len1)
    if len1 == n2:
            print((n2-n1)/n1)
    print((n1-n2)/n1)
    display(re[re["sequenceNo_y"].isnull()])
    display(re[re["sequenceNo_y"].isnull()]["SecurityID"].unique())
    display(re[re["sequenceNo_y"].isnull()]["TransactTime"].unique())
    display(re[(re["sequenceNo_y"].isnull()) & (re["idx_x"] == 1)])
    display(re[(re["sequenceNo_y"].isnull()) & (re["idx_x"] == 1)]["SecurityID"].unique())
    display(re[(re["sequenceNo_y"].isnull()) & (re["idx_x"] == 1)]["TransactTime"].unique())
    display(re[(re["sequenceNo_y"].isnull()) & (re["idx_x"] == 0)])
    display(re[(re["sequenceNo_y"].isnull()) & (re["idx_x"] == 0)]["SecurityID"].unique())
    display(re[(re["sequenceNo_y"].isnull()) & (re["idx_x"] == 0)]["TransactTime"].unique())
    
    display(re[re["sequenceNo_x"].isnull()])
    
    # OrderLog check    
    columns = ["TransactTime", "ApplSeqNum","Side",'OrderType', 'Price', 'OrderQty', "SecurityID"]
    ree = pd.merge(OrderLogSZ, OrderLogSZ1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
    n1 = ree["sequenceNo_x"].count()
    n2 = ree["sequenceNo_y"].count()
    len1 = len(ree)
    display(os.path.basename(os.path.basename(tpath2[i][0]).split("_")[1] + "_OrderLog Check_first 1000:"))
    print(n1)
    print(n2)
    print(len1)
    if len1 == n2:
            print((n2-n1)/n1)
    print((n1-n2)/n1)
    display(ree[np.isnan(ree["sequenceNo_y"])])
    display(ree[ree["sequenceNo_y"].isnull()]["SecurityID"].unique())
    display(ree[ree["sequenceNo_y"].isnull()]["TransactTime"].unique())
    display(ree[(np.isnan(ree["sequenceNo_y"])) & (ree["idx_x"] == 1)])
    display(ree[(np.isnan(ree["sequenceNo_y"])) & (ree["idx_x"] == 1)]["SecurityID"].unique())
    display(ree[(np.isnan(ree["sequenceNo_y"])) & (ree["idx_x"] == 1)]["TransactTime"].unique())
    display(ree[(np.isnan(ree["sequenceNo_y"])) & (ree["idx_x"] == 0)])
    display(ree[(np.isnan(ree["sequenceNo_y"])) & (ree["idx_x"] == 0)]["SecurityID"].unique())
    display(ree[(np.isnan(ree["sequenceNo_y"])) & (ree["idx_x"] == 0)]["TransactTime"].unique())
    
    display(ree[np.isnan(ree["sequenceNo_x"])])

    
#     # among stocks ordering check
#     TradeLogSZ1["tradeseq"] = TradeLogSZ1.index.values
#     OrderLogSZ1["orderseq"] = OrderLogSZ1.index.values
#     SZ = pd.concat([TradeLogSZ, OrderLogSZ]).sort_values(by=["sequenceNo", "clockAtArrival"])
#     SZ1 = pd.concat([TradeLogSZ1, OrderLogSZ1]).sort_values(by=["sequenceNo", "clockAtArrival"])
#     del TradeLogSZ
#     del TradeLogSZ1
#     del OrderLogSZ
#     del OrderLogSZ1
#     print(len(SZ["SecurityID"].unique()))
#     print(len(SZ1["SecurityID"].unique()))
    
#     # within stocks ordering check
#     columns = ["TransactTime", "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag", "TradePrice", "TradeQty",
#            "TradeMoney", "BidApplSeqNum", "OfferApplSeqNum", "Side", "OrderType", "Price", "OrderQty"]
#     stocks = SZ["SecurityID"].unique()
#     m = 0
#     diff = []
#     for i1 in stocks:
#         re1 = pd.merge(SZ[SZ["SecurityID"] == i1], SZ1[SZ1["SecurityID"] == i1], left_on=columns, right_on=columns, how="inner")
#         if (sorted(re1["sequenceNo_y"]) != re1["sequenceNo_y"]).any() == False:
#             m = m + 1
#         else:
#             diff.append(i1)
#     display(os.path.basename(os.path.basename(tpath2[i][0]).split("_")[1] + "_Ordering within stocks_first 1000:"))
#     if m == len(stocks):
#         print("SZE Trade + Order Data have same ordering within stocks")
#     print(m)
#     print(diff)
#     if len(diff)!=0:
#         for i1 in diff:
#             re1 = pd.merge(SZ[SZ["SecurityID"] == i1], SZ1[SZ1["SecurityID"] == i1], left_on=columns, right_on=columns, how="inner")
#             display(re1[sorted(re1["sequenceNo_y"]) != re1["sequenceNo_y"]])
#     del re1
#     del SZ
#     del SZ1
    
            
            
    TradeLog = pd.read_csv(tpath2[i][0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
    OrderLog = pd.read_csv(opath2[i][0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "Side", "OrderType", "Price",
                                                 "OrderQty"]]
    TradeLog1 = pd.read_csv(tpath1[i][0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
    OrderLog1 = pd.read_csv(opath1[i][0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "Side", "OrderType", "Price",
                                                 "OrderQty"]]
    stocks = OrderLog1["SecurityID"].unique()
    l1 = stocks[1000:]
    OrderLogSZ = OrderLog[OrderLog["SecurityID"].isin(l1)]
    OrderLogSZ1 = OrderLog1[OrderLog1["SecurityID"].isin(l1)]
    OrderLogSZ["idx"] = np.where(OrderLogSZ["SecurityID"].isin(ID_list.values), 1, 0)
    OrderLogSZ1["idx"] = np.where(OrderLogSZ1["SecurityID"].isin(ID_list.values), 1, 0)
    print(len(OrderLogSZ["SecurityID"].unique()))
    print(len(OrderLogSZ1["SecurityID"].unique()))
    del OrderLog
    del OrderLog1
    TradeLogSZ = TradeLog[TradeLog["SecurityID"].isin(l1)]
    TradeLogSZ1 = TradeLog1[TradeLog1["SecurityID"].isin(l1)]
    TradeLogSZ["idx"] = np.where(TradeLogSZ["SecurityID"].isin(ID_list.values), 1, 0)
    TradeLogSZ1["idx"] = np.where(TradeLogSZ1["SecurityID"].isin(ID_list.values), 1, 0)
    print(len(TradeLogSZ["SecurityID"].unique()))
    print(len(TradeLogSZ1["SecurityID"].unique()))
    del TradeLog
    del TradeLog1
    OrderLogSZ["OrderType"] = OrderLogSZ["OrderType"].apply(lambda x: str(x))
    OrderLogSZ1["OrderType"] = OrderLogSZ1["OrderType"].apply(lambda x: str(x))
    
    # TradeLog check
    TradeLogSZ["ExecType"] = TradeLogSZ["ExecType"].apply(lambda x: str(x))
    TradeLogSZ1["ExecType"] = TradeLogSZ1["ExecType"].apply(lambda x: str(x))
    columns = ["TransactTime","ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag","TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum","OfferApplSeqNum"]
    re = pd.merge(TradeLogSZ, TradeLogSZ1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
    n1 = re["sequenceNo_x"].count()
    n2 = re["sequenceNo_y"].count()
    len1 = len(re)
    display(os.path.basename(os.path.basename(tpath2[i][0]).split("_")[1] + "_TradeLog Check_second 1000:"))
    print(n1)
    print(n2)
    print(len1)
    if len1 == n2:
            print((n2-n1)/n1)
    print((n1-n2)/n1)
    display(re[re["sequenceNo_y"].isnull()])
    display(re[re["sequenceNo_y"].isnull()]["SecurityID"].unique())
    display(re[re["sequenceNo_y"].isnull()]["TransactTime"].unique())
    display(re[(re["sequenceNo_y"].isnull()) & (re["idx_x"] == 1)])
    display(re[(re["sequenceNo_y"].isnull()) & (re["idx_x"] == 1)]["SecurityID"].unique())
    display(re[(re["sequenceNo_y"].isnull()) & (re["idx_x"] == 1)]["TransactTime"].unique())
    display(re[(re["sequenceNo_y"].isnull()) & (re["idx_x"] == 0)])
    display(re[(re["sequenceNo_y"].isnull()) & (re["idx_x"] == 0)]["SecurityID"].unique())
    display(re[(re["sequenceNo_y"].isnull()) & (re["idx_x"] == 0)]["TransactTime"].unique())
    
    display(re[re["sequenceNo_x"].isnull()])
    
    # OrderLog check    
    columns = ["TransactTime", "ApplSeqNum","Side",'OrderType', 'Price', 'OrderQty', "SecurityID"]
    ree = pd.merge(OrderLogSZ, OrderLogSZ1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
    n1 = ree["sequenceNo_x"].count()
    n2 = ree["sequenceNo_y"].count()
    len1 = len(ree)
    display(os.path.basename(os.path.basename(tpath2[i][0]).split("_")[1] + "_OrderLog Check_second 1000:"))
    print(n1)
    print(n2)
    print(len1)
    if len1 == n2:
            print((n2-n1)/n1)
    print((n1-n2)/n1)
    display(ree[np.isnan(ree["sequenceNo_y"])])
    display(ree[ree["sequenceNo_y"].isnull()]["SecurityID"].unique())
    display(ree[ree["sequenceNo_y"].isnull()]["TransactTime"].unique())
    display(ree[(np.isnan(ree["sequenceNo_y"])) & (ree["idx_x"] == 1)])
    display(ree[(np.isnan(ree["sequenceNo_y"])) & (ree["idx_x"] == 1)]["SecurityID"].unique())
    display(ree[(np.isnan(ree["sequenceNo_y"])) & (ree["idx_x"] == 1)]["TransactTime"].unique())
    display(ree[(np.isnan(ree["sequenceNo_y"])) & (ree["idx_x"] == 0)])
    display(ree[(np.isnan(ree["sequenceNo_y"])) & (ree["idx_x"] == 0)]["SecurityID"].unique())
    display(ree[(np.isnan(ree["sequenceNo_y"])) & (ree["idx_x"] == 0)]["TransactTime"].unique())
    
    display(ree[np.isnan(ree["sequenceNo_x"])])
    
#     # among stocks ordering check
#     TradeLogSZ1["tradeseq"] = TradeLogSZ1.index.values
#     OrderLogSZ1["orderseq"] = OrderLogSZ1.index.values
#     SZ = pd.concat([TradeLogSZ, OrderLogSZ]).sort_values(by=["sequenceNo", "clockAtArrival"])
#     SZ1 = pd.concat([TradeLogSZ1, OrderLogSZ1]).sort_values(by=["sequenceNo", "clockAtArrival"])
#     del TradeLogSZ
#     del TradeLogSZ1
#     del OrderLogSZ
#     del OrderLogSZ1
#     print(len(SZ["SecurityID"].unique()))
#     print(len(SZ1["SecurityID"].unique()))
    
#     # within stocks ordering check
#     columns = ["TransactTime", "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag", "TradePrice", "TradeQty",
#            "TradeMoney", "BidApplSeqNum", "OfferApplSeqNum", "Side", "OrderType", "Price", "OrderQty"]
#     stocks = SZ["SecurityID"].unique()
#     m = 0
#     diff = []
#     for i1 in stocks:
#         re1 = pd.merge(SZ[SZ["SecurityID"] == i1], SZ1[SZ1["SecurityID"] == i1], left_on=columns, right_on=columns, how="inner")
#         if (sorted(re1["sequenceNo_y"]) != re1["sequenceNo_y"]).any() == False:
#             m = m + 1
#         else:
#             diff.append(i1)
#     display(os.path.basename(os.path.basename(tpath2[i][0]).split("_")[1] + "_Ordering within stocks_second 1000:"))
#     if m == len(stocks):
#         print("SZE Trade + Order Data have same ordering within stocks")
#     print(m)
#     print(diff)
#     if len(diff)!=0:
#         for i1 in diff:
#             re1 = pd.merge(SZ[SZ["SecurityID"] == i1], SZ1[SZ1["SecurityID"] == i1], left_on=columns, right_on=columns, how="inner")
#             display(re1[sorted(re1["sequenceNo_y"]) != re1["sequenceNo_y"]])
#     del re1
#     del SZ
#     del SZ1

[['\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_20200427_zs_92_01_day_data\\mdOrderLog_20200427_0832.csv'], ['\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_20200428_zs_92_01_day_data\\mdOrderLog_20200428_0832.csv']]
[['\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_20200427_zt_96_04_day_96data\\mdOrderLog_20200427_0835.csv'], ['\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_20200428_zt_96_04_day_96data\\mdOrderLog_20200428_0846.csv']]
[['\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_20200427_zs_92_01_day_data\\mdTradeLog_20200427_0832.csv'], ['\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_20200428_zs_92_01_day_data\\mdTradeLog_20200428_0832.csv']]
[['\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_20200427_zt_96_04_day_96data\\mdTradeLog_20200427_0835.csv'], ['\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_20200428_zt_96_04_day_96data\\mdTradeLog_20200428_0846.csv']]


C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


360
640
1000
1000


C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1000
1000


...

array([91503690, 93010260, 93010270, 93010290, 93010300, 93010310,
       93010320, 93010330, 93010340, 93010350, 93010360, 93010370,
       93010380, 93010400, 93104280, 93104290, 93104300, 93104310,
       93104370, 93104380, 93104390, 93104410, 93104420, 93104440],
      dtype=int64)

,clockAtArrival_x,sequenceNo_x,exchId_x,TransactTime,ApplSeqNum,SecurityID,Side,OrderType,Price,OrderQty,idx_x,clockAtArrival_y,sequenceNo_y,exchId_y,idx_y
118167,1.588037e+15,663790.0,2.0,91503690,95454,300431,1,2,20100,100,0.0,NaN,NaN,NaN,NaN
118168,1.588037e+15,663791.0,2.0,91503690,95455,2780,2,2,190600,500,0.0,NaN,NaN,NaN,NaN
118169,1.588037e+15,663792.0,2.0,91503690,95456,2780,2,2,193400,500,0.0,NaN,NaN,NaN,NaN
118170,1.588037e+15,663793.0,2.0,91503690,95457,2365,2,2,116000,1000,0.0,NaN,NaN,NaN,NaN
118176,1.588037e+15,663803.0,2.0,91503690,95463,300049,2,2,85900,800,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731487,1.588037e+15,4849604.0,2.0,93104430,782597,300730,2,2,131700,200,0.0,NaN,NaN,NaN,NaN
731490,1.588037e+15,4849615.0,2.0,93104430,834073,300667,1,2,243900,200,0.0,NaN,NaN,NaN,NaN
731527,1.588037e+15,4849762.0,2.0,93104440,782624,300603,2,2,261700,100,0.0,NaN,NaN,NaN,NaN
731530,1.588037e+15,4849773.0,2.0,93104440,782627,300681,1,2,331800,100,0.0,NaN,NaN,NaN,NaN


array([300431,   2780,   2365, 300049,   2760, 300119,   2472, 300806,
         2407, 300575, 300813,   2623, 300722,   2595, 300086, 300285,
         2482, 300636, 300334,   2233, 300628, 300211, 300374,   2953,
          987,   2661,   2118,   2316, 300535, 300391,   2227, 300315,
       300390, 300231, 300419, 300513, 300025, 300001, 300244,   2945,
         2689, 300204,   2476, 300452,   2793, 300399, 300466,   2771,
       300454,   2890, 300417, 300698, 300489, 300462,    886,   2699,
       300289, 300358,   2899,      4, 300604, 300529, 300581,   2460,
         2666, 300584,   2806,   2841, 300690, 300193, 300430, 300356,
          628, 300126, 300210, 300731, 300147,   2917, 300730, 300459,
       300040,   2831,    627, 300121,     20,   2311, 300470, 300389,
         2923, 300015,   2938, 300291,    518,     32, 300187,   2731,
       300206, 300810, 300479,    833, 300475, 300562, 300566, 300582,
         2829,   2641, 300655,    721, 300712,    880, 300135,   2702,
      

array([91503690, 93010260, 93010270, 93010280, 93010290, 93010300,
       93010310, 93010320, 93010340, 93010350, 93010360, 93010370,
       93010380, 93010400, 93010410, 93104290, 93104300, 93104310,
       93104370, 93104380, 93104390, 93104400, 93104410, 93104420,
       93104430, 93104440], dtype=int64)

,clockAtArrival_x,sequenceNo_x,exchId_x,TransactTime,ApplSeqNum,SecurityID,Side,OrderType,Price,OrderQty,idx_x,clockAtArrival_y,sequenceNo_y,exchId_y,idx_y
14865299,NaN,NaN,NaN,145959900,20010211,2307,2,2,56400,100,NaN,1.588057e+15,125136246.0,2.0,1.0
14865300,NaN,NaN,NaN,145959900,20010213,300419,1,2,54000,2000,NaN,1.588057e+15,125136250.0,2.0,0.0
14865301,NaN,NaN,NaN,145959900,19552279,2103,1,2,44100,1000,NaN,1.588057e+15,125136253.0,2.0,0.0
14865302,NaN,NaN,NaN,145959900,18597737,2870,2,2,207400,200,NaN,1.588057e+15,125136254.0,2.0,0.0
14865303,NaN,NaN,NaN,145959910,21050454,300404,2,2,229700,200,NaN,1.588057e+15,125136257.0,2.0,0.0
14865304,NaN,NaN,NaN,145959910,20010217,2168,1,2,65600,1600,NaN,1.588057e+15,125136258.0,2.0,1.0
14865305,NaN,NaN,NaN,145959910,20010218,300206,2,2,135500,200,NaN,1.588057e+15,125136260.0,2.0,0.0
14865306,NaN,NaN,NaN,145959920,20010221,300106,1,2,63500,3000,NaN,1.588057e+15,125136265.0,2.0,0.0
14865307,NaN,NaN,NaN,145959920,18597738,300169,2,2,61000,200,NaN,1.588057e+15,125136266.0,2.0,0.0
14865308,NaN,NaN,NaN,145959920,19552280,2848,1,2,118600,400,NaN,1.588057e+15,125136268.0,2.0,0.0
